# Introduction


In [1]:
import numpy as np
import pandas as pd
import pandapower as pp
import power_grid_model as pgm
from generate_fictional_dataset import generate_fictional_grid
import time

In [2]:
# fictional grid parameters

n_node_per_feeder = 3
n_feeder_small = 100
n_feeder_big = 2

cable_length_km_min = 0.8
cable_length_km_max = 1.2
load_p_w_max = 0.4e6 * 0.8
load_p_w_min = 0.4e6 * 1.2
pf = 0.95


# Big Network

In [3]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder_big,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp_net = fictional_dataset['pp_net']
pgm_dataset = fictional_dataset['pgm_dataset']
pgm_model = pgm.PowerGridModel(pgm_dataset)

## Symmetric

In [4]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow()
end = time.time()
print(f'Calculation time PGM: {end - start:4f} seconds')

Calculation time PGM: 0.111002 seconds


In [5]:
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
print(f'Calculation time PandaPoewr: {end - start:4f} seconds')

Calculation time PandaPoewr: 2.436998 seconds


In [6]:
np.abs(pp_net.res_bus['vm_pu'] - pgm_result['node']['u_pu']).max()

2.25409993093173e-06

In [7]:
np.abs(pp_net.res_line['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

6.637896740489813e-05

## Asymmetric

In [31]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False)
end = time.time()
print(f'Calculation time PGM: {end - start:4f} seconds')

Calculation time PGM: 0.000971 seconds


In [9]:
start = time.time()
pp.pf.runpp_3ph.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
print(f'Calculation time PandaPoewr: {end - start:4f} seconds')

Calculation time PandaPoewr: 0.120029 seconds


In [10]:
pp_net.line

,c_nf_per_km,r_ohm_per_km,x_ohm_per_km,c0_nf_per_km,r0_ohm_per_km,x0_ohm_per_km,max_i_ka,name,std_type,from_bus,to_bus,length_km,g_us_per_km,g0_us_per_km,df,parallel,type,in_service
0,400.0,0.063,0.103,660.0,0.156,0.1,1.0,None,630Al,0,1,1.019525,0.0,0.0,1.0,1,NaN,True
1,400.0,0.063,0.103,660.0,0.156,0.1,1.0,None,630Al,1,2,1.086076,0.0,0.0,1.0,1,NaN,True
2,400.0,0.063,0.103,660.0,0.156,0.1,1.0,None,630Al,2,3,1.041105,0.0,0.0,1.0,1,NaN,True
3,400.0,0.063,0.103,660.0,0.156,0.1,1.0,None,630Al,0,4,1.017953,0.0,0.0,1.0,1,NaN,True
4,400.0,0.063,0.103,660.0,0.156,0.1,1.0,None,630Al,4,5,0.969462,0.0,0.0,1.0,1,NaN,True
5,400.0,0.063,0.103,660.0,0.156,0.1,1.0,None,630Al,5,6,1.058358,0.0,0.0,1.0,1,NaN,True


In [11]:
pp_net

This pandapower network includes the following parameter tables:
   - bus (7 elements)
   - load (6 elements)
   - asymmetric_load (6 elements)
   - ext_grid (1 element)
   - line (6 elements)
 and the following results tables:
   - res_bus (7 elements)
   - res_line (6 elements)
   - res_ext_grid (1 element)
   - res_load (6 elements)
   - res_bus_3ph (7 elements)
   - res_line_3ph (6 elements)
   - res_ext_grid_3ph (1 element)
   - res_load_3ph (6 elements)
   - res_asymmetric_load_3ph (6 elements)

In [18]:
np.abs(pp_net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']].to_numpy() - pgm_result['node']['u_pu']).max()

0.0022608115779538807

In [20]:
np.abs(pp_net.res_line_3ph['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

0.06651633565380488

In [27]:
pp_net.res_line_3ph

,p_a_from_mw,q_a_from_mvar,p_b_from_mw,q_b_from_mvar,q_c_from_mvar,p_a_to_mw,q_a_to_mvar,p_b_to_mw,q_b_to_mvar,p_c_to_mw,...,i_b_ka,i_c_ka,i_n_from_ka,i_n_to_ka,i_n_ka,loading_a_percent,loading_b_percent,loading_c_percent,p_c_from_mw,loading_percent
0,0.788994,0.247438,0.723515,0.225476,0.244307,-0.787767,-0.250144,-0.722551,-0.228524,-0.778552,...,0.125243,0.135022,0.010675,0.010674,0.010675,13.663388,12.524267,13.502217,0.779736,13.663388
1,0.531869,0.166035,0.490877,0.152376,0.172250,-0.531279,-0.170042,-0.490407,-0.156564,-0.550067,...,0.085191,0.095630,0.009231,0.009231,0.009231,9.235683,8.519147,9.563022,0.550712,9.563022
2,0.262024,0.081542,0.242927,0.075221,0.090287,-0.261888,-0.086078,-0.242818,-0.079811,-0.288395,...,0.042325,0.050301,0.006925,0.006925,0.006925,4.567475,4.232477,5.030129,0.288570,5.030129
3,0.773215,0.242279,0.790027,0.248004,0.225832,-0.772057,-0.245136,-0.788796,-0.250705,-0.722580,...,0.136824,0.125265,0.010392,0.010392,0.010392,13.389522,13.682432,12.526492,0.723545,13.682432
4,0.473415,0.146977,0.486585,0.151373,0.143780,-0.473005,-0.150767,-0.486141,-0.155111,-0.463309,...,0.084461,0.080454,0.003477,0.003477,0.003477,8.215852,8.446148,8.045360,0.463697,8.446148
5,0.241205,0.074578,0.259241,0.080532,0.074911,-0.241091,-0.079243,-0.259100,-0.085162,-0.242084,...,0.045176,0.042191,0.003072,0.003071,0.003072,4.202600,4.517607,4.219124,0.242201,4.517607


In [28]:
pgm_result['line']['p_from'] * 1e-6

array([[0.40643225, 0.34116415, 0.39714888],
       [0.26943986, 0.22852366, 0.28823648],
       [0.12973788, 0.11065493, 0.15626438],
       [0.391652  , 0.40840001, 0.34209299],
       [0.23600246, 0.24914367, 0.22629563],
       [0.11740404, 0.13542711, 0.11839894]])

In [30]:
pp_net.res_asymmetric_load_3ph 


,p_a_mw,q_a_mvar,p_b_mw,q_b_mvar,p_c_mw,q_c_mvar
0,0.136662,0.044919,0.112439,0.036957,0.108605,0.035697
1,0.139550,0.045868,0.117775,0.038711,0.131792,0.043318
2,0.129704,0.042632,0.110635,0.036364,0.156211,0.051344
3,0.155353,0.051062,0.158922,0.052235,0.115594,0.037994
4,0.118498,0.038949,0.113599,0.037338,0.107807,0.035434
5,0.117378,0.038580,0.135388,0.044500,0.118372,0.038907


In [33]:
pgm_result['asym_load']['p'] * 1e-6

array([[0.13666202, 0.11243877, 0.10860465],
       [0.13954979, 0.11777466, 0.13179227],
       [0.12970429, 0.11063485, 0.15621141],
       [0.1553531 , 0.15892169, 0.11559361],
       [0.11849831, 0.11359935, 0.10780702],
       [0.11737821, 0.13538777, 0.11837178]])